In [ ]:
import import_ipynb

from keras.layers import Input, Dense, Lambda, Dropout, Concatenate, MultiHeadAttention, Layer, LayerNormalization, GlobalAveragePooling1D
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from IPython.display import display, Image, HTML, update_display
from tensorflow.keras.utils import plot_model
from keras.models import Model, Sequential
from keras.regularizers import L1L2
from threading import Thread, Lock
import keras.backend as K
import keras_tuner as kt
import tensorflow as tf
import numpy as np
import warnings
import datetime
import dataset
import tools
import users
import time
import uuid
import ads
import os

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

In [ ]:
root_directories = ["ADS16_Benchmark_part1", "ADS16_Benchmark_part2"]

In [ ]:
def load_user_and_ad_ftrs(root_directories, try_load_files=True, save_files=True):
    ad_ftrs_path = os.path.join("data", "ad_ftrs.npz")
    user_ftrs_and_ratings_path = os.path.join("data", "user_ftrs_and_ratings.npz")

    if try_load_files and os.path.exists(ad_ftrs_path):
        with np.load(ad_ftrs_path) as ad_ftrs_file:
            ad_ftrs = (ad_ftrs_file["ftrs1"], ad_ftrs_file["ftrs2"])
            num_categories = int(ad_ftrs_file["num_categories"])
    else:
        ad_ftrs, num_categories = ads.load_ftrs(root_directories)
        if save_files:
            np.savez(ad_ftrs_path, ftrs1=ad_ftrs[0], ftrs2=ad_ftrs[1], num_categories=num_categories)
    
    if try_load_files and os.path.exists(user_ftrs_and_ratings_path):
        with np.load(user_ftrs_and_ratings_path) as user_ftrs_and_ratings:
            user_ftrs = user_ftrs_and_ratings["user_ftrs"]
            ratings = user_ftrs_and_ratings["ratings"]
    else:
        raw_user_ftrs, ratings = users.load_raw_ftrs(root_directories)
        *_, user_ftrs = users.calculate_pca_ftrs(raw_user_ftrs)
        if save_files:
            np.savez(user_ftrs_and_ratings_path, user_ftrs=user_ftrs, ratings=ratings)
    
    return ad_ftrs, num_categories, user_ftrs, ratings

if __name__ == "__main__":
    ad_ftrs, num_categories, user_ftrs, ratings = load_user_and_ad_ftrs(root_directories)

In [ ]:
def get_user_ad_ftr_product(user_ftrs, ad_ftrs):
    return (tools.ftr_cartesian_product(user_ftrs, ad_ftrs[0]), tools.ftr_cartesian_product(user_ftrs, ad_ftrs[1])[-1])

if __name__ == "__main__":
    X = get_user_ad_ftr_product(user_ftrs, ad_ftrs)
    Y = tools.to_one_hot(ratings.flatten(), 1, 5)
    
    test_split = 0.2
    train_num = int((1-test_split) * X[0].shape[0])
    X_train, X_test = tools.split_all(X, train_num)
    Y_train, Y_test = Y[:train_num], Y[train_num:]


In [ ]:
def plot_distributions(model, title):
    rows = num_categories + 1
    cols = 3

    fig = plt.figure(constrained_layout=True, figsize=(cols*8, rows*6))
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        fig.suptitle(title, fontsize=20)
        fig.patch.set_facecolor("white")
        
        subfigs = fig.subfigures(rows, 1)

        pred = tools.from_one_hot(model.predict(X), 1, 5, mix=False).numpy()

        user_ftr_params = users.approximate_normal_params(user_ftrs.T)
        sim_user_ftrs = users.generate_synthetic_pca_ftrs(user_ftrs.shape[0], user_ftr_params)
        
        sim_pred = tools.from_one_hot(model.predict(get_user_ad_ftr_product(sim_user_ftrs, ad_ftrs)), 1, 5, mix=False).numpy()

        subfig = subfigs[0]
        subfig.suptitle("Overall Distributions")
        axs = subfig.subplots(1,3)
        
        pred_t = pred
        sim_pred_t = sim_pred
        # pred_t = np.clip(pred, 1, 5)
        # sim_pred_t = np.clip(sim_pred, 1, 5).astype(int)
        pred_t = np.round(pred_t).astype(int)
        sim_pred_t = np.round(sim_pred_t).astype(int)

        real_options = np.arange(1,6)
        pred_options = np.arange(min(real_options[0], np.min(pred_t)), max(real_options[-1], np.max(pred_t))+1)
        sim_pred_options = np.arange(min(real_options[0], np.min(sim_pred_t)), max(real_options[-1], np.max(sim_pred_t))+1)

        real_sizes = np.array([np.sum(ratings == i) for i in real_options])
        pred_sizes = np.array([np.sum(pred_t == i) for i in pred_options])
        sim_pred_sizes = np.array([np.sum(sim_pred_t == i) for i in sim_pred_options])

        i = 0
        axs[i].set_title("Ground Truth")
        axs[i].pie(real_sizes, labels=real_options, autopct='%1.1f%%', counterclock=False, startangle=90, explode=[0.01]*len(real_sizes))
        axs[i].legend()
        i += 1

        axs[i].set_title("Predicted (Real Users)")
        axs[i].pie(pred_sizes, labels=pred_options, autopct='%1.1f%%', counterclock=False, startangle=90, explode=[0.01]*len(pred_sizes))
        axs[i].legend()
        i += 1

        axs[i].set_title("Predicted (Simulated Users)")
        axs[i].pie(sim_pred_sizes, labels=sim_pred_options, autopct='%1.1f%%', counterclock=False, startangle=90, explode=[0.01]*len(sim_pred_sizes))
        axs[i].legend()
        i += 1


        for cat, subfig in enumerate(subfigs[1:]):
            subfig.suptitle(f"Category {cat}", fontsize=16)

            axs = subfig.subplots(1, cols)

            for col, ax in enumerate(axs):
                ax.plot()
                ax.set_xlabel(f"Rating")
                ax.set_ylabel(f"Probability Density")
                ax.xaxis.set_major_locator(MaxNLocator(integer=True))

            cat_mask = ad_ftrs[0][:,cat] == 1
            real_cat_ratings = ratings[:,cat_mask]
            pred_cat_ratings = pred.reshape(ratings.shape)[:,cat_mask]
            sim_cat_ratings = sim_pred.reshape(ratings.shape)[:,cat_mask]

            def get_bins(x):
                bins = np.arange(np.floor(min(np.min(x), np.min(real_cat_ratings))),np.ceil(max(np.max(x), np.max(real_cat_ratings)))+2,1) - 0.5
                bin_centers = 0.5*(bins[1:]+bins[:-1])
                return bins, bin_centers

            real_bins, real_bin_centers = get_bins(real_cat_ratings)
            pred_bins, pred_bin_centers = get_bins(pred_cat_ratings)
            sim_bins, sim_bin_centers = get_bins(sim_cat_ratings)

            i = 0

            axs[i].set_title(f"Ground Truth")
            for user in real_cat_ratings:
                heights, _ = np.histogram(user, bins=real_bins, density=True)
                axs[i].plot(real_bin_centers, heights, c="blue", alpha=0.1)
            i += 1
            
            axs[i].set_title(f"Predicted (Real Users)")
            for user in pred_cat_ratings:
                heights, _ = np.histogram(user, bins=pred_bins, density=True)
                axs[i].plot(pred_bin_centers, heights, c="red", alpha=0.1)
            i += 1
            
            axs[i].set_title(f"Predicted (Simulated Users)")
            for user in sim_cat_ratings:
                heights, _ = np.histogram(user, bins=sim_bins, density=True)
                axs[i].plot(sim_bin_centers, heights, c="green", alpha=0.1)
            i += 1
        
        plt.close()
    return fig

# if __name__ == "__main__":
#     display(plot_distributions(create_model(), "Rating Distributions"))


In [ ]:
def call_with_args_kwargs(func, args, kwargs):
    return func(*args, **kwargs)

class DisplayableCallback(Callback):
    def __init__(self, name, create_display_immediately=True, **kwargs):
        super().__init__(**kwargs)
        self.name = name
        self.display_id = f"{self.name}-{str(uuid.uuid1())}"
        if create_display_immediately:
            self.create_display()
    
    def create_display(self):
        display(HTML(self.name), display_id=self.display_id)
    
    def update_display(self, obj):
        update_display(obj, display_id=self.display_id)

class Printer(DisplayableCallback):
    def __init__(self, **kwargs):
        super().__init__("Epoch Data", **kwargs)
        
    def on_epoch_end(self, epoch, logs=None):
        header = f"Epoch {epoch}"
        lines = []
        if logs is not None:
            for k, v in logs.items():
                lines.append(f"{k}={v:.5f}")
        self.update_display(HTML(f"<h3>{header}</h3>"+"<br/>".join(lines)))
        return super().on_epoch_end(epoch, logs=logs)
    

class ModelDisplayer(DisplayableCallback):
    def __init__(self, **kwargs):
        super().__init__("Model Topology", **kwargs)
    
    def on_epoch_end(self, epoch, logs=None):
        model_plot_filename = "data/tmp.png"
        plot_model(self.model, to_file=model_plot_filename, show_shapes=True, show_layer_activations=True, show_layer_names=True)
        self.update_display(Image(model_plot_filename))
        os.unlink(model_plot_filename)
        return super().on_epoch_end(epoch, logs=logs)


class DistributionPlotter(DisplayableCallback):
    def __init__(self, **kwargs):
        super().__init__("Rating Distributions", **kwargs)
        self.lock = Lock()
        
    def on_epoch_end_(self, epoch, logs=None):
        with self.lock:
            self.update_display(plot_distributions(self.model, f"Rating Distributions: Epoch {epoch}"))

    def on_epoch_end(self, epoch, logs=None):
        if not self.lock.locked():
            Thread(target=self.on_epoch_end_, args=(epoch, logs)).start()
        return super().on_epoch_end(epoch, logs=logs)
    
    def plot_now(self):
        with self.lock:
            self.update_display(plot_distributions(self.model, f"Rating Distributions"))
            

if __name__ == "__main__":
    logs_dir="user_ad_interaction_model_logs"
    checkpoints_dir = os.path.join(logs_dir, "checkpoints")
    filepath = os.path.join(checkpoints_dir, "model-{epoch:06d}-{val_loss:06f}.hdf5")
    checkpoint = ModelCheckpoint(filepath, monitor=["val_loss"], save_weights_only=True, verbose=0, save_best_only=False, mode="min")

    log_dir = os.path.join(logs_dir, "fit", datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    

In [ ]:
def rating_mae(y_true, y_pred):
    r_true = tools.from_one_hot(y_true, 1, 5, mix=False)
    r_pred = tools.from_one_hot(y_pred, 1, 5, mix=False)
    return tf.keras.metrics.mean_absolute_error(r_true, r_pred)

def mixed_rating_mae(y_true, y_pred):
    r_true = tools.from_one_hot(y_true, 1, 5, mix=True)
    r_pred = tools.from_one_hot(y_pred, 1, 5, mix=True)
    return tf.keras.metrics.mean_absolute_error(r_true, r_pred)


class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


def create_model(hp=None):
    ad_ftrs, _, user_ftrs, ratings = load_user_and_ad_ftrs(root_directories)

    inputs_1 = Input((user_ftrs.shape[1]+ad_ftrs[0].shape[1]), name="flat_input")
    inputs_2 = Input((ad_ftrs[1].shape[1:]), name="text_input")

    input_dropout_rate = 0.5 if hp is None else hp.Choice("input_dropout", values=[0.0, 0.1, 0.3, 0.5])
    subnet_1 = inputs_1 if input_dropout_rate == 0 else Dropout(input_dropout_rate, name="flat_input_dropout")(inputs_1)
    subnet_2 = inputs_2 if input_dropout_rate == 0 else Dropout(input_dropout_rate, name="text_input_dropout")(inputs_2)

    l1 = 1e-2 if hp is None else hp.Choice("l1", values=[1e-2, 1e-3, 1e-4])
    l2 = 1e-2 if hp is None else hp.Choice("l2", values=[1e-2, 1e-3, 1e-4])
    reg = L1L2(l1=l1, l2=l2)

    num_hidden_layers = 2 if hp is None else hp.Int("num_hidden_layers", 1, 3)
    for i in range(num_hidden_layers):
        units = 32 if hp is None else hp.Int(f"units_{i}", min_value=32, max_value=128, step=32)
        subnet_1 = Dense(units, activation="relu", kernel_regularizer=reg, name=f"hidden_{i}")(subnet_1)
        dropout_rate = 0.5 if hp is None else hp.Choice(f"dropout_{i}", values=[0.0, 0.1, 0.3, 0.5])
        if dropout_rate != 0:
            subnet_1 = Dropout(dropout_rate, name=f"hidden_dropout_{i}")(subnet_1)

    num_heads = 2 if hp is None else hp.Int("num_attn_heads", min_value=1, max_value=8, step=1)
    ff_dim = 32 if hp is None else hp.Int("ff_dim", min_value=32, max_value=128, step=32)
    dropout_rate = 0.5 if hp is None else hp.Choice("transformer_dropout_1", values=[0.0, 0.1, 0.3, 0.5])
    subnet_2 = TransformerBlock(embed_dim=ad_ftrs[1].shape[-1], num_heads=num_heads, ff_dim=ff_dim, rate=dropout_rate)(subnet_2)
    subnet_2 = GlobalAveragePooling1D()(subnet_2)
    dropout_rate = 0.5 if hp is None else hp.Choice("transformer_dropout_2", values=[0.0, 0.1, 0.3, 0.5])
    if dropout_rate != 0:
        subnet_2 = Dropout(dropout_rate)(subnet_2)

    output = Concatenate(name="combined_subnets")([subnet_1, subnet_2])
    units_final = 32 if hp is None else hp.Int("units_final", min_value=32, max_value=128, step=32)
    output = Dense(units_final, kernel_regularizer=reg, name="hidden_final", activation="relu")(output)
    dropout_rate = 0.5 if hp is None else hp.Choice("dropout_final", values=[0.0, 0.1, 0.3, 0.5])
    if dropout_rate != 0:
        output = Dropout(dropout_rate)(output)
    output = Dense(5, kernel_regularizer=reg, name="output", activation="softmax")(output)

    model = Model(inputs=[inputs_1, inputs_2], outputs=[output])
    learning_rate = 1e-2 if hp is None else hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy", rating_mae, mixed_rating_mae]
    )

    return model

class MyTuner(kt.Hyperband):
    def run_trial(self, trial, *args, **kwargs):
        kwargs['batch_size'] = trial.hyperparameters.Int('batch_size', 32, 256, step=32)
        return super().run_trial(trial, *args, **kwargs)

if __name__ == "__main__":

    tuner = MyTuner(
            create_model,
            objective=kt.Objective("val_rating_mae", direction="min"),
            max_epochs=100,
            factor=3,
            hyperband_iterations=5,
            directory=os.path.join(logs_dir, "hypertraining"),
            project_name="user_ad_interaction"
        )
    
    # print("EXAMPLE MODEL --- HYPERPARAMS NOT TUNED")
    # model = create_model()
    # model.summary()
    # c = ModelDisplayer()
    # c.model = model
    # c.on_epoch_end(0)

    # c = DistributionPlotter()
    # c.model = model
    # c.plot_now()


In [ ]:
if __name__ == "__main__":
    epochs = 100
    
    if not os.path.isdir(checkpoints_dir):
        os.makedirs(checkpoints_dir)
    
    
    try:
        tf.get_logger().setLevel("ERROR")

        params = {
            "verbose": 0,
            "shuffle": True,
            "epochs": epochs
        }


        display(HTML("<h2>HyperTuning</h2>"))
        tuner.search(
            X_train,
            Y_train,
            **params,
            validation_split=0.2,
            callbacks=[tensorboard_callback, Printer(), ModelDisplayer(), EarlyStopping(monitor='val_loss', mode="min", patience=5)]
        )


        display(HTML("<h2>Found Optimal Model!</h2>"))
        num_trials = 1

        tuner.results_summary(num_trials=num_trials)
        print("\n")
        best_hps=tuner.get_best_hyperparameters(num_trials=num_trials)[0]

        model = tuner.hypermodel.build(best_hps)
        batch_size = best_hps.Int('batch_size', 32, 256, step=32)
        
        # model = create_model()
        # batch_size = 32

        model.summary()

        model.fit(
            X_train,
            Y_train,
            **params,
            batch_size=batch_size,
            validation_data=(X_test,Y_test),
            callbacks=[tensorboard_callback, checkpoint, ModelDisplayer(), Printer(), d:=DistributionPlotter()]#, EarlyStopping(monitor='val_loss', mode="min", patience=5)]
        )

        d.plot_now()

    except KeyboardInterrupt:
        pass